In [1]:
import ibis
from ibis import _
from cng.utils import set_secrets, s3_client


In [2]:
@ibis.udf.scalar.builtin
def h3_cell_to_parent(str, int) -> int:
    ...


In [3]:

import ibis
from ibis import _
from cng.utils import set_secrets, s3_client

con = ibis.duckdb.connect("tmp.db", extensions=['httpfs', 'spatial', 'h3'])
set_secrets(con)

tracts = con.read_parquet("s3://public-census/year=2022/tracts-hex-z10.parquet")

In [4]:
#svi = con.read_parquet("s3://public-data/social-vulnerability/2022/SVI2022_US_tract.parquet").rename(geom = "Shape").select(['STATE','COUNTY','FIPS', 'geom'])
#ca_tracts = con.read_parquet("s3://public-social-vulnerability/2022-tracts-h3-z10.parquet").join(svi, "FIPS").to_parquet("s3://public-census/year=2022/tracts-hex-z10.parquet")
con.raw_sql("SET enable_progress_bar = false")
con.raw_sql("SET memory_limit = '62GB'")


In [5]:
states = tracts.select("STATE").distinct().execute()["STATE"]


In [6]:
m = s3_client()
stuff = m.list_objects("public-census", "year=2022/state=", recursive=True)

finished = [i.object_name for i in stuff if not i.is_dir]


In [7]:
for state in states:
    print(state)
    st = tracts.filter(_.STATE == state)
    counties = st.select("COUNTY").distinct().execute()["COUNTY"]
    for county in counties:
        if not county == "Coconino County":
            if not f"year=2022/state={state}/county={county}/tracts.parquet" in finished:
                print("   \t" + county)
                sf = (st
                  .filter(_.COUNTY == county)
                  .mutate(h10 = _.h10.unnest().cast(str).upper())
               #   .mutate(h0 = h3_cell_to_parent(_.h10, 0).cast(str).upper())
                  .drop("STATE")
                  .drop("COUNTY")
                  .to_parquet(f"s3://public-census/year=2022/state={state}/county={county}/tracts.parquet")
                 )
#sf.head().execute()

Arizona
Iowa
   	Dubuque County
   	Guthrie County
   	Jackson County
   	Lucas County
   	Mills County
   	Poweshiek County
   	Wright County
   	Cerro Gordo County
   	Clay County
   	Clayton County
   	Dallas County
   	Davis County
   	Howard County
   	Jones County
   	Marshall County
   	Palo Alto County
   	Pocahontas County
   	Polk County
   	Shelby County
   	Story County
   	Adams County
   	Allamakee County
   	Appanoose County
   	Benton County
   	Carroll County
   	Clinton County
   	Emmet County
   	Franklin County
   	Henry County
   	Jasper County
   	Lee County
   	Linn County
   	Pottawattamie County
   	Union County
   	Washington County
   	Audubon County
   	Black Hawk County
   	Buchanan County
   	Cass County
   	Clarke County
   	Crawford County
   	Greene County
   	Humboldt County
   	Marion County
   	Scott County
   	Warren County
   	Winneshiek County
   	Worth County
   	Adair County
   	Bremer County
   	Cedar County
   	Des Moines County
   	Grundy Cou

OutOfMemoryException: Out of Memory Error: failed to allocate data of size 16.0 MiB (57.7 GiB/57.7 GiB used)

In [ ]:
h0 = sf.select('h0').distinct().execute()['h0'][0]
tile = f"s3://public-gbif/hex/h0={h0}/*"  # could be multiple tiles
gbif = con.read_parquet(tile)
#gbif

In [ ]:
sf_birds = gbif.join(sf, 'h10').filter(_['class'] == "Aves").select(['species', 'FIPS', 'geom']).distinct()

In [ ]:
sf_birds.group_by(_.FIPS).agg(n = _.count()).head().execute()